### 通过 MovieTweetings 创建推荐系统：最热门的推荐内容

你已经创建了将在这节课剩余部分创建推荐系统时使用的必要列，下面我们开始创建推荐系统的第一个步骤吧。

首先，使用以下代码读取将在这节课中一直使用的库和两个数据集。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1.如何查找最热门的电影

对于此 notebook，我们只有一个任务。我们的任务是无论用户是谁，我们都需要根据最热门的项目提供一个推荐列表。

对于此任务，我们将根据以下标准判断什么“最热门”：

* 平均评分最高的电影被视为最佳电影
* 如果评分一样，则评分数量更多的电影更好
* 电影如果评分不足 5 条，则不能被视为最佳电影
* 如果电影的平均评分和评分数量都一样，那么根据最近的评分判断排名

根据这些标准，此 notebook 的目标是获取 **user_id** 并返回 **n_top** 推荐。以下函数将作为所有未来推荐系统的框架。

In [3]:
movies.shape

(31245, 35)

In [4]:
def create_ranked_df(movies, reviews):
        '''
        INPUT
        movies - the movies dataframe
        reviews - the reviews dataframe
        
        OUTPUT
        ranked_movies - a dataframe with movies that are sorted by highest avg rating, more reviews, 
                        then time, and must have more than 4 ratings
        '''
        
        # Pull the average ratings and number of ratings for each movie
        movie_ratings = reviews.groupby('movie_id')['rating']
        avg_ratings = movie_ratings.mean()
        num_ratings = movie_ratings.count()
        last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
        last_rating.columns = ['last_rating']

        # Add Dates
        rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        # merge with the movies dataset
        movie_recs = movies.set_index('movie_id').join(rating_count_df)

        # sort by top avg rating and number of ratings
        ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        # for edge cases - subset the movie list to those with only 5 or more reviews
        ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
        
        return ranked_movies
    

def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time

    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''

    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies
        



有了上述三个标准后，你将能够编写上述函数。如果你对自己的答案很有信心了，可以对照我们的解答检查你的函数代码。下个页面会提供代码演示，当然，你也可以查看此 workspace 中的解答 notebook。

In [5]:
# Top 20 movies recommended for id 1
ranked_movies = create_ranked_df(movies, reviews) # only run this once - it is not fast

recs_20_for_1 = popular_recommendations('1', 20, ranked_movies)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations('53968', 5, ranked_movies)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('70000', 100, ranked_movies)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35, ranked_movies)

In [10]:
### You Should Not Need To Modify Anything In This Cell
# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**注意：**这并不是确定“最高评分”电影的唯一方式。如果是跟踪热门新闻或社会事件，则需要创建一个从当前时间开始的时间期限，然后从最近的期限内提取报道。至于什么方式最好，我们需要自己判断。

如果你发现没有人关注你的最热门推荐内容了，那么就需要寻找新的推荐方式，这节课的后续部分将介绍这方面的知识。


### 第二部分：添加过滤器

创建返回 **n_top** 电影的函数后，我们来完善下此函数。添加作为电影**年份 year**和**类型 genre**过滤器的语句。  

在以下单元格中调整现有函数，将**年份**和**类型**参数设为**字符串列表**。然后，从提供的年份和类型列表（作为 `or` 条件）中过滤出最终结果。如果没有提供列表，则不应用过滤器。

你可以根据需要调整其他输入，从而检索你想要的最终结果。

请在我们的测试函数中编写一些测试。下面这行代码会根据指定的年份和类型过滤器为用户 1 返回前 20 部热门电影。你的代码返回的结果一样吗？

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [ ]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    REDO THIS DOC STRING
    
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # create top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies
